In [21]:
# Download metadata to ./data/weather
import os
import urllib


# URLs for metadata
metadata_urls = {
    'station_inventory': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt', 
    'states': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-states.txt', 
    'stations': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 
    'countries': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt'
}

# Output dir
directory = './data/weather'
if not os.path.exists(directory):
    os.makedirs(directory)

uo = urllib.URLopener()

for k, u in metadata_urls.items():
    uo.retrieve(u, './data/weather/'+k+'.txt')

In [1]:
# Download daily weather data summary by year
import requests
import gzip
import io
import csv

# URL for yearly data 
# Uncomment the yr_base_url and yrs to run the script

# yr_base_url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/'
# yrs = range(1997, 2015+1)

for yr in yrs:
    yr_url = ''.join([yr_base_url, str(yr), '.csv.gz'])
    
    print 'downloading file for: %d...' % (yr)
    resp = requests.get(yr_url, stream=True)
    csv_gz = resp.content
    f = io.BytesIO(csv_gz)
    print 'download complete'
    
    with gzip.GzipFile(fileobj=f, mode='rb') as fh, gzip.open('./data/weather/'+str(yr)+'.csv.gz', 'wb+') as fout:
        print 'filtering file...'
        for line in fh:
            if 'US' == line[:2]:
                fout.write(line)
        print 'finished writing file for: %d...\n' % (yr)

downloading file for: 2013...
download complete
filtering file...
finished writing file for: 2013...

downloading file for: 2014...
download complete
filtering file...
finished writing file for: 2014...

downloading file for: 2015...
download complete
filtering file...
finished writing file for: 2015...



In [13]:
# Filter US Stations to a separate file
stations_file = './data/weather/stations.txt'
us_stations = './data/weather/us_stations.txt'

with open(stations_file, 'rb') as fin, open(us_stations, 'wb+') as fout:
    for line in fin:
        if 'US' == line[:2]:
            fout.write(line)

# yrs = range(1997, 1997+1)
# print yrs


# for yr in yrs:
#     with gzip.open('./data/weather/'+str(yr)+'.csv.gz', mode='rb') as fh:
#         for line in fh:
#             print line


In [43]:
# Find the closest station for each fire 
import requests
import gzip
import io
import csv
import sys
from collections import defaultdict


wildfires = './data/wildfires_10k_sample.csv'
us_stations = './data/weather/us_stations.txt'

# load us stations latitude and longitude in a dictionary
us_latlon = defaultdict(str)
with open(us_stations, 'rb') as fus:
    for line in fus:
        station_code = line[:11].replace(' ', '')
        lat = line[11:20].replace(' ', '')
        lon = line[20:30].replace(' ', '')
        us_latlon[lat+','+lon] = station_code.upper()

# print us_latlon
hs = Haversine()

us_latlon_file = './data/weather/fire_weather_station.csv'
with open(wildfires, 'rb') as fwild, open(us_latlon_file, 'wb+') as fout:
    for line in fwild:
        line = line.strip().split(',')
        fire_code = line[0]
        fire_lat, fire_lon = map(float, [line[-2], line[-1]])
        
        closest_station = ['None', 0., 0., sys.maxsize, 0.]
        
        
        for key in us_latlon.keys():
            s_lat, s_lon = map(float, key.split(','))
            
            dist = hs.distance((fire_lat, fire_lon), (s_lat, s_lon))
            if dist <= closest_station[3]:
                closest_station[0] = us_latlon[key]
                closest_station[1] = s_lat
                closest_station[2] = s_lon                
                closest_station[3] = dist

        
        closest_station[4] = hs.bearing((fire_lat, fire_lon), closest_station[2])
        out_rec = [fire_code, fire_lat, fire_lon] + closest_station
        fout.write(','.join(map(str, out_rec))+'\n')

In [44]:
! head ./data/weather/fire_weather_station.csv

829373,30.7047,-89.3525,USR0000MHCK,30.6153,-89.4133,7.15834253377,144.335122146
1809170,38.6,-95.28,US1KSFR0013,38.6036,-95.2691,0.639144173568,139.377929358
431856,34.94978909,-84.35280409,USC00095917,34.9528,-84.3214,1.79109584946,144.9363249
710286,31.50096327,-89.10201294,USC00226626,31.4833,-89.0333,4.22948788372,144.174691615
880502,31.2072,-84.1956,USC00091500,31.1903,-84.2036,1.2601239291,146.289264691
1058606,35.25,-81.05,US1NCGS0009,35.249,-81.0335,0.933833718477,146.380419306
532620,35.13398,-82.70145,US1NCTR0024,35.1442,-82.7316,1.8445960404,145.62582336
675489,45.97181249,-91.35116212,USW00094973,46.0261,-91.4442,5.83349557081,136.995876083
709012,30.83872356,-89.09707676,US1MSST0005,30.856,-89.1249,2.03745403541,144.389308128
274347,45.97,-89.8877,USC00474320,45.95,-89.8833,1.39832841092,137.527232708


In [2]:
# Helper class for claculating distance between two lat, lon
# https://www.movable-type.co.uk/scripts/latlong.html

from math import *

class Haversine:
    def __init__(self, radius=3959.87433):
        self.RADIUS = radius
        
    def distance(self, src, dest):
        ''' Returns distance in miles
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dest)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        
        return self.RADIUS * c
    
    def bearing(self, src, dest):
        '''Returns compass degrees
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dst)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        y = sin(dlon) * cos(lon2)
        x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
        bearing = atan2(y, x)
        bearing = degrees(bearing)
        return (bearing + 360) % 360
    
    
if __name__ == '__main__':
    dst = (-94.5930, -39.1230)
    src = (-94.4839, -39.1561)
    h = Haversine()
    print h.distance(src, dst)
    print h.bearing(src, dst)

7.54238357394
166.75532027


In [17]:
dst = (34.0, -118.0)
src = (34.0, -73.0)
h = Haversine()
print h.distance(src, dst)
print h.bearing(src, dst)

2556.79350884
67.7541634922


In [216]:
# Extract date from wildfires sqlite
import sqlite3
import pandas as pd

conn = sqlite3.connect('./data/wf.db')
df = pd.read_sql_query("SELECT OBJECTID, FIRE_NAME, FIRE_YEAR, DISCOVERY_DATE, CONT_DATE From Fires", conn)
df = df.sort_values('FIRE_YEAR')
df.head()
df.DAYS = df.CONT_DATE - df.DISCOVERY_DATE
df['DAYS'] = df.DAYS

In [217]:
# Impute CONT_DATE and DAYS values for FIRES with these fields missing
avg_control_days = np.mean(df.DAYS[~pd.isnull(df.DAYS)])
df.DAYS[pd.isnull(df.DAYS)] = avg_control_days
df.CONT_DATE[pd.isnull(df.CONT_DATE)] = df.DISCOVERY_DATE[pd.isnull(df.CONT_DATE)] + df.DAYS[pd.isnull(df.CONT_DATE)]

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [239]:
# Convert datetime 
import datetime

epoch = pd.to_datetime(0, unit='s').to_julian_date()
df.START_DATE = pd.to_datetime(df.DISCOVERY_DATE - epoch, unit='D')
df.START_DATE = pd.to_datetime(df.START_DATE, format='%Y%m%d').dt.date

In [240]:
df.END_DATE = pd.to_datetime(df.CONT_DATE - epoch, unit='D')
df.END_DATE = pd.to_datetime(df.END_DATE, format='%Y%m%d').dt.date

In [241]:
df['START_DATE'] = df.START_DATE
df['END_DATE'] = df.END_DATE
df.head()

,OBJECTID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DAYS,START_DATE,END_DATE
691277,691278,None,1992,2448671.5,2.448672e+06,0.0,1992-02-19,1992-02-19
761559,761560,None,1992,2448801.5,2.448803e+06,NaN,1992-06-28,1992-06-29
761560,761561,None,1992,2448801.5,2.448803e+06,NaN,1992-06-28,1992-06-29
761561,761562,None,1992,2448815.5,2.448817e+06,NaN,1992-07-12,1992-07-13
761562,761563,None,1992,2448819.5,2.448821e+06,NaN,1992-07-16,1992-07-17


In [248]:
df['DAYS'] = df.DAYS
df = df.sort_values('FIRE_YEAR')

In [249]:
# drop FIRE_NAMES
df.drop(['FIRE_NAME'], axis=1)

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DAYS,START_DATE,END_DATE
691277,691278,1992,2448671.5,2.448672e+06,0.000000,1992-02-19,1992-02-19
962069,962070,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962068,962069,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962067,962068,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962066,962067,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962065,962066,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962064,962065,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
962062,962063,1992,2448681.5,2.448683e+06,1.233722,1992-02-29,1992-03-01
206009,206010,1992,2448776.5,2.448776e+06,0.000000,1992-06-03,1992-06-03
206010,206011,1992,2448776.5,2.448776e+06,0.000000,1992-06-03,1992-06-03


In [250]:
df.to_csv('./data/fire_dates.csv', encoding='utf-8')

In [254]:
# Read the fire_stations file to join fire data with closes stations
fire_stations = pd.read_csv('./data/weather/fire_weather_stations.csv.gz', compression='gzip', header=None)
fire_stations.columns = ['OBJECTID', 'FIRE_LAT', 'FIRE_LON', 'STATION_ID', 'STAT_LAT', 'STAT_LON', 'DIST', 'BEARING']
fire_stations.head()

,OBJECTID,FIRE_LAT,FIRE_LON,STATION_ID,STAT_LAT,STAT_LON,DIST,BEARING
0,829373,30.704700,-89.352500,USR0000MHCK,30.6153,-89.4133,7.158343,144.335122
1,1809170,38.600000,-95.280000,US1KSFR0013,38.6036,-95.2691,0.639144,139.377929
2,431856,34.949789,-84.352804,USC00095917,34.9528,-84.3214,1.791096,144.936325
3,710286,31.500963,-89.102013,USC00226626,31.4833,-89.0333,4.229488,144.174692
4,880502,31.207200,-84.195600,USC00091500,31.1903,-84.2036,1.260124,146.289265


In [263]:
# Join the fire+dates data with weather stations and distance
combined = df.set_index('OBJECTID').join(fire_stations.set_index('OBJECTID'))
combined.head()

# Only keep fires where we have a STATION_ID 
# Write to CSV
combined_subset = combined[~pd.isnull(combined.STATION_ID)]
combined_subset.to_csv('./data/weather/fire_dates_stations.csv.gz', 
                                                 header=True, compression='gzip')

In [378]:
combined_subset['FIRE_ID'] = combined_subset.index
combined_subset.head()

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DAYS,START_DATE,END_DATE,FIRE_LAT,FIRE_LON,STATION_ID,STAT_LAT,STAT_LON,DIST,BEARING,FIRE_ID
OBJECTID,,,,,,,,,,,,,,,
961744,None,1992,2448643.5,2.448645e+06,1.233722,1992-01-22,1992-01-23,28.530000,-82.460000,US1FLHN0019,28.5290,-82.4629,0.189167,147.875111,961744
962436,None,1992,2448728.5,2.448730e+06,1.233722,1992-04-16,1992-04-17,27.480000,-81.300000,US1FLHL0012,27.4050,-81.2461,6.148003,148.706933,962436
1114005,USSERY,1992,2448775.5,2.448777e+06,1.233722,1992-06-02,1992-06-03,38.656111,-122.843056,US1CASN0114,38.6605,-122.8639,1.165135,135.066496,1114005
1390149,None,1992,2448701.5,2.448702e+06,0.000000,1992-03-20,1992-03-20,33.178500,-83.892200,USC00091453,33.1500,-83.8833,2.035905,145.774615,1390149
997402,None,1992,2448716.5,2.448718e+06,1.233722,1992-04-04,1992-04-05,35.145000,-76.791700,USC00318370,35.1500,-76.7667,1.454440,148.662592,997402


In [421]:
def get_date_range(s, e):
    s = pd.date_range(end=s, periods=5)[0]
    e = pd.date_range(start=e, periods=5)[-1]
    return pd.date_range(s, e, format='%Y-%m-%d')

print get_date_range('1997-01-01', '1997-01-10')

DatetimeIndex(['1996-12-28', '1996-12-29', '1996-12-30', '1996-12-31',
               '1997-01-01', '1997-01-02', '1997-01-03', '1997-01-04',
               '1997-01-05', '1997-01-06', '1997-01-07', '1997-01-08',
               '1997-01-09', '1997-01-10', '1997-01-11', '1997-01-12',
               '1997-01-13', '1997-01-14'],
              dtype='datetime64[ns]', freq='D')


In [429]:
# Expand fire date ranges
# Output each year to a separate file
from copy import copy

yrs = range(1997, 2015+1)
out_sub = []
for yr in yrs:
    subs = combined_subset[combined_subset.FIRE_YEAR == yr]
    
    for idx, row in subs.iterrows():
        date_range = get_date_range(row['START_DATE'], row['END_DATE'])
        
        for d in date_range:
            new_row = copy(row)
            new_row['OBS_DATE'] = d.strftime('%Y-%m-%d')
            out_sub.append(new_row)
            new_row = None
   

    out_sub = pd.DataFrame(out_sub)
    out_sub.sort_values('FIRE_ID')
    out_sub.to_csv('./data/weather/' + str(yr) + '_fire_dates.csv.gz', compression='gzip')
    out_sub = []

In [430]:
! ls -alh ./data/weather/*_fire_date.csv.gz

ls: ./data/weather/*_fire_date.csv.gz: No such file or directory


In [184]:
# # Append dates to fires
# import requests
# import gzip
# import io
# import csv
# import sys
# from collections import defaultdict


# fsf = './data/weather/fire_weather_stations.csv.gz'
# fire_stations = defaultdict(str)


# with gzip.open(fsf, mode='rb') as f:
#     for line in f:
#         line = line.strip().split(',')
#         fire_stations[line[0]] = line[3]
    
# print fire_stations

,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DAYS,START_DATE,END_DATE,FIRE_LAT,FIRE_LON,STATION_ID,STAT_LAT,STAT_LON,DIST,BEARING
OBJECTID,,,,,,,,,,,,,,
501277,None,1997,2450552.5,2.450554e+06,1.233722,1997-04-14,1997-04-15,46.725503,-93.711454,US1MNCW0017,46.7460,-93.9616,11.933158,135.770128
501210,None,1997,2450567.5,2.450569e+06,1.233722,1997-04-29,1997-04-30,46.589943,-94.205953,US1MNCW0004,46.6177,-94.1909,2.047197,135.692880
501043,None,1997,2450557.5,2.450559e+06,1.233722,1997-04-19,1997-04-20,45.988627,-94.211135,USC00216479,45.9667,-94.1000,5.548680,136.036025
1443779,None,1997,2450542.5,2.450544e+06,1.233722,1997-04-04,1997-04-05,43.720959,-89.666855,USC00479319,43.6089,-89.7667,9.214049,138.841127
501116,None,1997,2450556.5,2.450558e+06,1.233722,1997-04-18,1997-04-19,45.793402,-94.718187,US1MNTD0003,45.9048,-94.6506,8.358320,135.989270


In [293]:
import os.path

url_prefix = './data/weather/'
url_suffix = '.csv.gz'

os.path.isfile('./data/weather/1992.csv.gz')

start_yr = 1997
current_yr = 0
is_current_yr_file = False

file_url = ''

file_handle = None
file_contents = None


out_df = pd.DataFrame()
def read_gzip_file(file_url):
    return pd.read_csv(file_url, header=None, compression='gzip')

# m_out_df = pd.DataFrame()

out_df = read_gzip_file('./data/weather/1997.csv.gz')

,0,1,2,3,4,5,6,7
0,USW00024061,19970101,TMAX,172,NaN,NaN,M,NaN
1,USW00024061,19970101,TMIN,-6,NaN,NaN,M,NaN
2,USW00024229,19970101,TMAX,139,NaN,NaN,0,2400.0
3,USW00024229,19970101,TMIN,100,NaN,NaN,0,2400.0
4,USW00024229,19970101,PRCP,122,NaN,NaN,0,2400.0


In [297]:
out_df.columns = ['STAT_ID', 'OBS_DATE', 'KEYWORD', 'VAL', '01', '02', '03', '04']

In [306]:
out_df.DATE = pd.to_datetime(out_df.OBS_DATE, format='%Y-%m-%d').dt.date

In [308]:
out_df['DATE'] = out_df.DATE
out_df.head()

,STAT_ID,OBS_DATE,KEYWORD,VAL,01,02,03,04,DATE
0,USW00024061,19970101,TMAX,172,NaN,NaN,M,NaN,1970-01-01
1,USW00024061,19970101,TMIN,-6,NaN,NaN,M,NaN,1970-01-01
2,USW00024229,19970101,TMAX,139,NaN,NaN,0,2400.0,1970-01-01
3,USW00024229,19970101,TMIN,100,NaN,NaN,0,2400.0,1970-01-01
4,USW00024229,19970101,PRCP,122,NaN,NaN,0,2400.0,1970-01-01


In [317]:
mgdf = pd.merge(combined_1997, out_df, how='left', left_on=['STATION_ID', 'START_DATE'], right_on = ['STAT_ID', 'DATE'])
mgdf.head()

,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DAYS,START_DATE,END_DATE,FIRE_LAT,FIRE_LON,STATION_ID,...,BEARING,STAT_ID,OBS_DATE,KEYWORD,VAL,01,02,03,04,DATE
0,None,1997,2450552.5,2.450554e+06,1.233722,1997-04-14,1997-04-15,46.725503,-93.711454,US1MNCW0017,...,135.770128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,1997,2450567.5,2.450569e+06,1.233722,1997-04-29,1997-04-30,46.589943,-94.205953,US1MNCW0004,...,135.692880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,1997,2450557.5,2.450559e+06,1.233722,1997-04-19,1997-04-20,45.988627,-94.211135,USC00216479,...,136.036025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,1997,2450542.5,2.450544e+06,1.233722,1997-04-04,1997-04-05,43.720959,-89.666855,USC00479319,...,138.841127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,1997,2450556.5,2.450558e+06,1.233722,1997-04-18,1997-04-19,45.793402,-94.718187,US1MNTD0003,...,135.989270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
mgdf.columns

Index([u'FIRE_NAME', u'FIRE_YEAR', u'DISCOVERY_DATE', u'CONT_DATE', u'DAYS',
       u'START_DATE', u'END_DATE', u'FIRE_LAT', u'FIRE_LON', u'STATION_ID',
       u'STAT_LAT', u'STAT_LON', u'DIST', u'BEARING', u'STAT_ID', u'OBS_DATE',
       u'KEYWORD', u'VAL', u'01', u'02', u'03', u'04', u'DATE'],
      dtype='object')

In [315]:
for i, fire in combined_1997[combined_1997.FIRE_YEAR == 1997].iterrows():
    fsdate = str(fire['START_DATE'])
    fedate = str(fire['END_DATE'])
    for j, weather in out_df.iterrows():
        if fire.STATION_ID != weather.STAT_ID:
            continue 
        wdate = str(weather['DATE'])
        print fire.STATION_ID, weather

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [ ]:
for idx, row in combined_1997.iterrows():
    row_yr = int(row['FIRE_YEAR'])
    
    if row_yr < start_yr:
        # We don't have weather data before 1997
        continue
        
    if current_yr == row_yr:
        # do something
        pass
    
    else: 
        if file_handle:
            file_handle.close()
            print 'closing file_handle'
        current_yr = row_yr
        file_url = url_prefix + str(current_yr) + url_suffix
        
        print 'opening file_handle'
        file_handle = open(file_url, 'rb')

if file_handle:
    print 'closing file_handle'
    file_handle.close()

In [9]:
import requests
import gzip
import io
import csv
import sys
from collections import defaultdict

out_rec = defaultdict(object, {
    'fire_id': '', 
    'station_id': '', 
    'observation_date': '', 
    'TMAX': '', 
    'TMIN': '', 
    'PRCP': '', 
    'SNOW': '', 
    'SNWD': '', 
    'WDFG': '', 
    'WSFG': '', 
    'WT03': '', 
    'WT08': '', 
    'WT16': '', 
})

required_station = ''
required_start_date = ''
required_end_date = ''
current_station = ''
current_date = ''

fsf = './data/weather/fire_weather_stations.csv.gz'
fire_stations = defaultdict(str)


with gzip.open(fsf, mode='rb') as f:
    for line in f:
        line = line.strip().split(',')
        fire_stations[line[0]] = line[3]
    
print fire_stations

defaultdict(<type 'str'>, {'221540': 'US1COME0118', '477977': 'USC00168715', '880576': 'US1FLGD0005', '1124372': 'US1CATH0023', '549307': 'US1NJHN0032', '1619562': 'US1NCRT0008', '1763647': 'USC00416118', '1008175': 'US1NCBR0012', '1471443': 'USC00502872', '897562': 'USC00347732', '1816315': 'USC00092475', '1745702': 'US1CAOR0015', '1795336': 'USC00049099', '1452757': 'US1MOWS0002', '123852': 'USR0000AODE', '635778': 'USC00411413', '533995': 'US1NCCL0026', '978800': 'US1FLPT0009', '1345891': 'USC00308946', '1472182': 'US1ORCC0005', '1082508': 'USW00093242', '501116': 'US1MNTD0003', '1359055': 'US1FLBV0028', '1540052': 'US1TXST0005', '665119': 'US1VAPN0002', '300257': 'USC00027726', '1199694': 'US1PANM0002', '1590375': 'USC00254485', '1378260': 'US1GAAP0001', '420627': 'USW00013861', '413693': 'US1FLSW0007', '738806': 'US1SCFL0001', '1134667': 'USC00046035', '1715363': 'USC00084412', '732878': 'US1SCHM0011', '1349956': 'USC00306459', '1527720': 'USC00419836', '1073443': 'US1CAFR0006', '